# ReOpt

- API structure: https://developer.nrel.gov/api/reopt/stable/help?API_KEY=DEMO_KEY
- Models.py: https://github.com/NREL/REopt_API/blob/master/reoptjl/models.py

In [1]:
import pandas as pd
import shutil
import os, sys, json
from src.post_and_poll import get_api_results
from utils import import_df, import_json, export_json, plotly_stacked, parse_dispatch_series, setup

# to silence warnigns
# InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised.
import urllib3
urllib3.disable_warnings()

pd.options.plotting.backend='plotly'

API_KEY = '0bgdMrMpcFEfzIuYLRFpEfPeUqabOQKo5RagyRPV' 

In [2]:
def create_post(cfg,array,solar_kw,batt_kw,batt_h):
    post = {
        "Site": {
            "longitude": 45.505890265478996, 
            "latitude": 9.160999024421262
        },
        "PV": {
            "min_kw":solar_kw,              "max_kw":solar_kw,            
            "production_factor_series":import_df(cfg.solar_file,array,resamp=cfg.solar_resamp,scale=cfg.solar_scaler),
            "array_type": 0,
            "can_net_meter":False,
            "installed_cost_per_kw":0,
        },
        "ElectricLoad": {
            'loads_kw':import_df(cfg.load_file,cfg.load_col,resamp=cfg.load_resamp)
        },
        "ElectricStorage": {
            "min_kw":batt_kw,               "max_kw":batt_kw,
            "min_kwh":batt_kw*batt_h,       "max_kwh":batt_kw*batt_h,
            "installed_cost_per_kw":0,      "installed_cost_per_kwh":0,
            "soc_min_fraction":0.0,         "soc_max_fraction":1.0,     "soc_init_fraction":0.0,
            "can_grid_charge":True,
        },

        "ElectricTariff": {
            "urdb_response":import_json(cfg.tariff_file),
            'wholesale_rate':[cfg.energy_price_sell_constant]*8760,
            #'wholesale_rate':import_df('data/PG&E NBT EEC Values 2024 Vintage.csv','Price',resamp='1h'),
            #'urdb_label': '5c817fcf7bece2299eceb39b', # General Service Time-of-use Electric Vehicle Charging - Demand Metered:TOU-EV-4 (50kV - 220 kV) - Closed Rate            
        },
        "ElectricUtility":{
            "net_metering_limit_kw":0,
            
        },
        "Financial": {
            "analysis_years": 1,
        }
    }
    
    export_json(post,cfg.outdir)
    
    return post

def run_reopt(post, print_results=False):

    outputs_file_name = "results_file"
    root_url = "https://developer.nrel.gov/api/reopt/stable" # /stable == /v3 
    
    try:
        api_response = get_api_results(post=post, 
                                    API_KEY=API_KEY, 
                                    api_url=root_url, 
                                    results_file= f'outputs/{outputs_file_name}.json', 
                                    run_id=None)
    except:
        print('API request failed')
        
    if api_response is not None:
        cost =      api_response["outputs"]["ElectricTariff"]["year_one_energy_cost_before_tax"]
        revenue =   api_response["outputs"]["ElectricTariff"]["year_one_export_benefit_before_tax"]
        netcost = cost - revenue

        if print_results:    
            print('Status = ',              api_response["status"])
            print("Energy cost ($) = ",     cost)
            print('Energy revenue ($) = ',  revenue)
            print('Net cost ($) = ',        netcost)
            print('PV Size (kW) = ',        api_response["outputs"]["PV"]["size_kw"])
            if "ElectricStorage" in api_response["outputs"].keys():
                print('Storage Size (kW-kwh) = ',api_response["outputs"]["ElectricStorage"]["size_kw"],'-',api_response["outputs"]["ElectricStorage"]["size_kwh"])
    else:
        print('API request failed')
        cost = pd.NA
        revenue = pd.NA
        netcost = pd.NA
    return api_response, cost, netcost

def compare_to_s20(cfg,solar_kw:int,batt_kw:int,batt_h:int,print_results=False):
    cost_red,cost_red_pct,netcost_red = [], [], []
    post = create_post(cfg,'0w90',solar_kw,batt_kw,batt_h)
    _, cost_base, netcost_base = run_reopt(post,print_results)
    for array in cfg.arrays:
        post = create_post(cfg,f'{array}w90',solar_kw,batt_kw,batt_h)
        _, cost, netcost = run_reopt(post,print_results)
        cost_red.append(        cost_base - cost)
        cost_red_pct.append(    1 - cost/abs(cost_base))
        netcost_red.append(     netcost_base - netcost)
        if print_results:
            print(f'\nCost Reduction {array} ($) = {cost_base-cost:.2f} ({100*(1-cost/abs(cost_base)):.1f}%)')
            print(f'NetCost Reduction {array} ($) = {netcost_base-netcost:.2f} ({100*(1-netcost/abs(netcost_base)):.1f}%)\n')
    return cost_base, cost_red, cost_red_pct, netcost_red

# Setup

In [3]:
cfg = setup('reopt.yaml')

# Single run

In [4]:
post = create_post(cfg,'25w90',125,25,2)
api_response, cost, netcost = run_reopt(post)
print(cost, netcost)

5335.47 2808.4700000000003


# Grid search

In [5]:
cols = ['SolarKW','BattKW','BattH','BaseCost']
df = pd.DataFrame([],columns=cols   + [f'Reduc_{x}w90' for x in cfg.arrays]\
                                    + [f'Reduc%_{x}w90' for x in cfg.arrays]\
                                    + [f'NetReduc_{x}w90' for x in cfg.arrays])

#best_cost_red_pct = 0
#best_resp = None

if cfg.test:
    cfg.solar_kws   = [cfg.solar_kws[0]]
    cfg.batt_kws    = [cfg.batt_kws[0]]
    cfg.batt_hs     = [cfg.batt_hs[0]]

for solar_kw in cfg.solar_kws:
    for batt_kw in cfg.batt_kws:
        for batt_h in cfg.batt_hs:
            basecost, cost_red, cost_red_pct, netcost_red = compare_to_s20(cfg,solar_kw,batt_kw,batt_h,print_results=False)
            df.loc[len(df)] = [solar_kw,batt_kw,batt_h,basecost]+cost_red+cost_red_pct+netcost_red
            df.to_csv(cfg.outdir+'results.csv',index=False)
            print(df.iloc[:,:7])
            
            # for x,r,n in zip(cost_red_pct,resp,arrays):
            #     if x > best_cost_red_pct:
            #         best_cost_red_pct = x
            #         best_resp = r
            #         df.iloc[-1,:].to_csv(f'outputs/best_result_{n}.csv')
            
df

   SolarKW  BattKW  BattH  BaseCost  Reduc_25w90  Reduc%_25w90  NetReduc_25w90
0    100.0     0.0    1.0   8314.78       152.21      0.018306         -140.79
   SolarKW  BattKW  BattH  BaseCost  Reduc_25w90  Reduc%_25w90  NetReduc_25w90
0    100.0     0.0    1.0   8314.78       152.21      0.018306         -140.79
1    100.0     0.0    2.0   8314.78       152.21      0.018306         -140.79
   SolarKW  BattKW  BattH  BaseCost  Reduc_25w90  Reduc%_25w90  NetReduc_25w90
0    100.0     0.0    1.0   8314.78       152.21      0.018306         -140.79
1    100.0     0.0    2.0   8314.78       152.21      0.018306         -140.79
2    100.0    25.0    1.0   7339.73       150.33      0.020482         -148.67
API request failed


UnboundLocalError: cannot access local variable 'api_response' where it is not associated with a value

# Plot

In [ ]:
# dispatch = parse_dispatch_series(api_response)
# f = plotly_stacked(dispatch,solar='pv',load='load',batt='batt',utility='grid',soc='soc',theme='plotly_dark')